In [1]:
# libraries
import ndjson
import json
import pandas as pd
import numpy as np
import altair as alt
import gzip
import random
import csv
import re

import matplotlib.pyplot as plt
import seaborn as sns
from pywaffle import Waffle

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.util import mark_negation
nltk.download("opinion_lexicon")
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import opinion_lexicon

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sa = SentimentIntensityAnalyzer()
sa.lexicon

from imblearn.under_sampling import RandomUnderSampler 

%matplotlib inline

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\hewin\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hewin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hewin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hewin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# open csv and create list with just the reviews
with open('small_corpus_vader.csv', 'r') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    reviews = []
    for row in readCSV:
        reviews.append(row)

    print(reviews[0:3])

[['', 'ratings', 'sentiment', 'reviews'], ['1', '1.0', '-0.9511', "code didn't work, got me a refund."], ['2', '1.0', '0.0', 'these do not work at all, all i get is static and they came with nothing in the box to give any help.']]


In [3]:
reviews = pd.DataFrame(reviews)
reviews = reviews.rename(columns = reviews.iloc[0]).drop(reviews.index[0])
reviews = reviews.iloc[:, [1,2,3]]
reviews

,ratings,sentiment,reviews
1,1.0,-0.9511,"code didn't work, got me a refund."
2,1.0,0.0,"these do not work at all, all i get is static ..."
3,1.0,0.4019,well let me start by saying that when i first ...
4,1.0,-0.9988,"Dont waste your money, you will just end up us..."
5,1.0,0.3657,i ordered this and said it was tested and work...
...,...,...,...
4495,5.0,0.8849,"Nice long micro USB cable, battery lasts a lon..."
4496,5.0,0.743,I've been having a great time with this game. ...
4497,5.0,0.8519,d
4498,5.0,0.0,"Really pretty, funny, interesting game. Works ..."


In [38]:
reviews['sentiment  classification'] = ['Positive' if v > 0.2 else 'Neutral' if v >= -0.2 and v <= 0.2 else 'Negative' for v in reviews['sentiment'].astype(float)]
reviews

,ratings,sentiment,reviews,class,sentiment classification
1,1.0,-0.9511,"code didn't work, got me a refund.",None,Negative
2,1.0,0.0,"these do not work at all, all i get is static ...",None,Neutral
3,1.0,0.4019,well let me start by saying that when i first ...,None,Positive
4,1.0,-0.9988,"Dont waste your money, you will just end up us...",None,Negative
5,1.0,0.3657,i ordered this and said it was tested and work...,None,Positive
...,...,...,...,...,...
4495,5.0,0.8849,"Nice long micro USB cable, battery lasts a lon...",None,Positive
4496,5.0,0.743,I've been having a great time with this game. ...,None,Positive
4497,5.0,0.8519,d,None,Positive
4498,5.0,0.0,"Really pretty, funny, interesting game. Works ...",None,Neutral


In [39]:
reviews['ratings  classification'] = ['Positive' if v == 5 else 'Neutral' if v == 2 or v == 3 or v == 4 else 'Negative' for v in reviews['ratings'].astype(float)]
reviews

,ratings,sentiment,reviews,class,sentiment classification,ratings classification
1,1.0,-0.9511,"code didn't work, got me a refund.",None,Negative,Negative
2,1.0,0.0,"these do not work at all, all i get is static ...",None,Neutral,Negative
3,1.0,0.4019,well let me start by saying that when i first ...,None,Positive,Negative
4,1.0,-0.9988,"Dont waste your money, you will just end up us...",None,Negative,Negative
5,1.0,0.3657,i ordered this and said it was tested and work...,None,Positive,Negative
...,...,...,...,...,...,...
4495,5.0,0.8849,"Nice long micro USB cable, battery lasts a lon...",None,Positive,Positive
4496,5.0,0.743,I've been having a great time with this game. ...,None,Positive,Positive
4497,5.0,0.8519,d,None,Positive,Positive
4498,5.0,0.0,"Really pretty, funny, interesting game. Works ...",None,Neutral,Positive
